In [1]:
using Combinatorics
using LinearAlgebra
using LaTeXStrings
using Chaos
using Statistics
using CSV
using DataFrames
using GLMakie

┌ Info: Precompiling Chaos [0aea4425-8685-4cf8-ab93-d00eaca7e4b3]
└ @ Base loading.jl:1423
┌ Warning: Package Chaos does not have LinearAlgebra in its dependencies:
│ - If you have Chaos checked out for development and have
│   added LinearAlgebra as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with Chaos
└ Loading LinearAlgebra into Chaos from project dependency, future warnings for Chaos are suppressed.


In [2]:
Na = 90; # number of atoms
Ls = 3; # number of wells

### The particle conserved basis

In [3]:
Hbs = integer_partition(N=Na, L=Ls);

In [4]:
Lb = length(Hbs) # the dimension of the Hilbert space

4186

In [5]:
Lb_shouldbe = binomial(Na+Ls-1, Na)

4186

### The hopping Hamiltonian

In [6]:
HLs = zeros(Float64, Lb, Lb)
for i in 1:Ls-1
    HLs += hopping_pair(m=i, Na=Na, Hbs=Hbs)
end

In [7]:
Hhop = HLs + HLs';

### The onsite interaction

In [8]:
Honsite = zeros(Float64, Lb, Lb);

In [9]:
for j in 1:Lb
    for k in 1:Lb
        if j == k
            sum = 0
            for l in 1:Ls
                sum += Hbs[j][l]^2 - Hbs[j][l]
            end
            Honsite[j,k] += sum
        end
    end
end

### The tilt potential

In [10]:
function Htit(γ::Float64)
    Htit = zeros(Float64, Lb, Lb)
    for j in 1:Lb
        for k in 1:Lb
            if j == k
                for l in 1:Ls
                    Htit[j,k] += -(l-(Ls+1)/2)*γ*Hbs[j][l]
                end
            end

        end
    end
    return Htit
end

Htit (generic function with 1 method)

### Soft-core interactions

In [11]:
Λ(delta::Int64, d::Float64, C6::Float64, R::Float64) = C6 / ((delta)^6*d^6 + R^6)

Λ (generic function with 1 method)

In [12]:
# Here 'd' is the lattice constant.
function Hsc(d::Float64, C6::Float64, R::Float64)
    Hsc = zeros(Float64, Lb, Lb)
    for j in 1:Lb
        for k in 1:Lb
            if j == k
                for l in 1:Ls
                    for m in 1:Ls
                        if l-m == 0
                            Hsc[j,k] += 2*Λ(0,d,C6,R)*Hbs[j][l]*Hbs[j][m]
                        elseif abs(m-l) == 1
                            Hsc[j,k] += Λ(1,d,C6,R)*Hbs[j][l]*Hbs[j][m]
                        elseif abs(m-l) == 2
                            Hsc[j,k] += Λ(2,d,C6,R)*Hbs[j][l]*Hbs[j][m]
                        else
                            continue
                        end
                    end
                end
            end
        end
    end
    return Hsc
end

Hsc (generic function with 1 method)

In [13]:
function Hsc_new(;Λ::Float64)
    Hsc = zeros(Float64, Lb, Lb)
    for j in 1:Lb
        for k in 1:Lb
            if j == k
                for l in 1:Ls
                    for m in 1:Ls
                        if l-m == 0
                            Hsc[j,k] += 2*Λ*Hbs[j][l]*Hbs[j][m]
                        elseif abs(m-l) == 1
                            Hsc[j,k] += Λ/2*Hbs[j][l]*Hbs[j][m]
                        elseif abs(m-l) == 2
                            Hsc[j,k] += Λ/8*Hbs[j][l]*Hbs[j][m]
                        else
                            continue
                        end
                    end
                end
            end
        end
    end
    return Hsc
end

Hsc_new (generic function with 1 method)

### Construction of the full Hamiltonian

In [14]:
function Htot(;γ::Float64, J::Float64, g::Float64, d::Float64, C6::Float64, R::Float64)
    Htot = Htit(γ) - J*Hhop + (g/2)*Honsite + (1/2)*Hsc(d,C6,R)
    return Htot
end

Htot (generic function with 1 method)

In [15]:
function Htot_new(;γ::Float64, J::Float64, g::Float64, Λ::Float64)
    Htot = Htit(γ) - J*Hhop + (g/2)*Honsite + (1/2)*Hsc_new(Λ=Λ)
    return Htot
end

Htot_new (generic function with 1 method)

### Plots

In [16]:
J = 1.0;
g = 1.0;
γs = 0.0:1.0:10.0;
Λs = -5.0:0.5:5.0

-5.0:0.5:5.0

In [17]:
Dict1 = Dict{Float64, Vector{Vector{Float64}}}(i=>[] for i in Λs);

In [ ]:
for Λ in Λs
    for γ in γs
        H = Htot_new(γ=γ, J=1.0, g=1.0, Λ=Λ)
        eigs = eigvals(H)
        push!(Dict1[Λ], eigs)
    end
end

In [ ]:
# Plot the diagrams w.r.t γ
fontsize_theme = Theme(fontsize = 45)
set_theme!(fontsize_theme)

for Λ in Λs
    eigs = Dict1[Λ];
    eigs_mat = zeros(Float64, length(γs), Lb);
    for i in 1:length(γs)
        eigs_mat[i,:] = eigs[i]
    end
    
    f = Figure(resolution=(800,800))
    ax = Axis(f[1,1], xlabel=L"\gamma", ylabel=L"\epsilon")
    ax.title = "Λ=$Λ"
    for i in 1:Lb
        lines!(ax, γs, eigs_mat[:,i])
    end
    save("/Users/tianyiyan/Desktop/Chaos/juliapics/spectrum/Λ=$Λ.png",f)
end